In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# 나스닥 상위 40개 종목 리스트
nasdaq_tickers = [
    "AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "ADBE", "AVGO",
    "AMD", "INTC", "CSCO", "CMCSA", "PEP", "QCOM", "TXN", "AMAT", "COST", "MRVL",
    "ADP", "VRTX", "ISRG", "INTU", "PYPL", "LRCX", "SBUX", "GILD", "ATVI", "REGN",
    "MDLZ", "BKNG", "SNPS", "ADSK", "ORLY", "KLAC", "FTNT", "PANW", "TEAM", "ZS"
]

# CCI 계산 함수 정의
def calculate_cci(data, n=20):
    """
    특정 종목의 데이터를 받아서 CCI 계산 수행.
    """

    print(data.head())

    data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3  # Typical Price
    data['SMA_TP'] = data['TP'].rolling(window=n).mean()
    data['MAD'] = data['TP'].rolling(window=n).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    data['CCI'] = (data['TP'] - data['SMA_TP']) / (0.015 * data['MAD'])
    return data

# CCI 상태 라벨링 함수 정의
cci_bins = np.arange(-200, 220, 10)  # 상태 범위
def label_state(value, bins):
    return np.digitize(value, bins, right=True)

# 전이 확률 계산에 필요한 변수 정의
num_states = len(cci_bins)
all_states = [str(i) for i in range(num_states)]
state_to_index = {state: idx for idx, state in enumerate(all_states)}

cci_scope_bins = np.arange(-100, 100, 2)  # 상태 범위
def label_state_scope(value, bins):
    return np.digitize(value, bins, right=True)

num_scope_bins = len(cci_scope_bins)
all_scope_bins = [str(i) for i in range(len(cci_scope_bins))]
scope_to_index = {scope: idx for idx, scope in enumerate(all_scope_bins)}

# 전이 확률 행렬 초기화
transition_matrix = np.zeros((num_states, num_states, num_scope_bins))

# 데이터프레임 초기화
data_frames = []

# 모든 종목 데이터를 하나씩 다운로드하여 처리
for ticker in nasdaq_tickers:
    print(f"Downloading data for {ticker}...")
    datay = yf.download(ticker, start="2017-01-01", end="2023-01-01")

    if datay.empty:
        print(f"No data for {ticker}, skipping...")
        continue

    if isinstance(datay.columns, pd.MultiIndex):
      datay.columns = [col[0] for col in datay.columns.values]

    # 리스트에 추가
    data_frames.append(datay)

# 데이터프레임 결합
combined_data = pd.concat(data_frames)

# 인덱스 재설정 (날짜만 인덱스로 사용)
combined_data.reset_index(inplace=True)
combined_data.set_index('Date', inplace=True)

data = combined_data

# CCI 계산
data = calculate_cci(data)

# CCI 상태 라벨링
data['CCI_State'] = data['CCI'].apply(lambda x: label_state(x, cci_bins))

# 상태 및 다음 상태 정의
data['State'] = data['CCI_State'].astype(str)
data['Next_State'] = data['CCI_State'].shift(-1).astype(str)

# Scope 및 Scope Label 계산
data['scope'] = data['CCI_State'] - data['CCI_State'].shift(1)
data['scope_label'] = data['scope'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

# 상태 전이 및 기울기(Scope_Label) 쌍별 count 계산
transition_counts = data.groupby(['State', 'Next_State', 'scope_label']).size().reset_index(name='actual_count')

# 전이 확률 행렬 업데이트
for _, row in transition_counts.iterrows():
    start_state, next_state, scope_label, count = row['State'], row['Next_State'], row['scope_label'], row['actual_count']
    if pd.notna(start_state) and pd.notna(next_state) and pd.notna(scope_label):
        try:
            # 문자열로 변환
            start_state = str(int(float(start_state)))
            next_state = str(int(float(next_state)))
            scope_label = str(int(float(scope_label)))

            # 상태 및 기울기 인덱스가 정의되어 있는 경우에만 업데이트
            if (start_state in state_to_index and
                next_state in state_to_index and
                scope_label in scope_to_index):

                # Transition matrix를 상태 및 기울기 인덱스를 기준으로 업데이트
                transition_matrix[state_to_index[start_state], state_to_index[next_state], scope_to_index[scope_label]] += count
        except ValueError:
            pass


print(transition_counts)

# 전이 확률 계산
row_sums = transition_matrix.sum(axis=1, keepdims=True)
transition_matrix = np.divide(
    transition_matrix, row_sums, out=np.zeros_like(transition_matrix), where=row_sums != 0
)



# 전이 확률 계산 함수
def calculate_transition_probability(start_state, end_cci_state, steps):
    """
    특정 시작 상태에서 종료 상태까지 전이 확률 계산.
    """
    P_t = np.zeros(num_states)
    P_t[state_to_index[start_state]] = 1
    for _ in range(steps):
        P_t = np.dot(P_t, transition_matrix)
    return P_t

# 테스트: 특정 상태에서 종료 상태까지의 확률 계산
start_state = "10"  # 시작 상태
end_cci_state = "1"  # 종료 상태
steps = 3  # 몇 단계 후의 확률을 계산할지

probability = calculate_transition_probability(start_state, end_cci_state, steps)
#print(f"전이 확률 (5단계 후): {probability}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...

No data for ATVI, skipping...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


                           Adj Close      Close       High        Low  \
Date                                                                    
2017-01-03 00:00:00+00:00  26.891962  29.037500  29.082500  28.690001   
2017-01-04 00:00:00+00:00  26.861860  29.004999  29.127501  28.937500   
2017-01-05 00:00:00+00:00  26.998465  29.152500  29.215000  28.952499   
2017-01-06 00:00:00+00:00  27.299450  29.477501  29.540001  29.117500   
2017-01-09 00:00:00+00:00  27.549498  29.747499  29.857500  29.485001   

                                Open     Volume  
Date                                             
2017-01-03 00:00:00+00:00  28.950001  115127600  
2017-01-04 00:00:00+00:00  28.962500   84472400  
2017-01-05 00:00:00+00:00  28.980000   88774400  
2017-01-06 00:00:00+00:00  29.195000  127007600  
2017-01-09 00:00:00+00:00  29.487499  134247600  
     State Next_State  scope_label  actual_count
0        0        0.0         30.0             2
1        0        0.0         33.0      

ValueError: shapes (42,100) and (42,42,100) not aligned: 100 (dim 1) != 42 (dim 1)

In [ ]:
import pandas as pd
import numpy as np

# transition_matrix_total을 DataFrame으로 변환
transition_matrix_df = pd.DataFrame(
    transition_matrix_total.reshape(-1),  # 1D로 변환
    columns=["Probability"]
)

# 멀티인덱스 생성 (start_state, next_state, scope_label)
index_arrays = np.indices(transition_matrix_total.shape)
multi_index = pd.MultiIndex.from_arrays(
    [index_arrays[0].flatten(), index_arrays[1].flatten(), index_arrays[2].flatten()],
    names=["Start_State", "Next_State", "Scope_Label"]
)
transition_matrix_df.index = multi_index

# DataFrame 형태로 저장
output_file_path = "/content/transition_matrix_total.xlsx"
transition_matrix_df.to_excel(output_file_path)


In [ ]:
import pandas as pd

# transition_counts 데이터를 엑셀로 저장
output_file_path = "/content/transition_counts.xlsx"
transition_counts.to_excel(output_file_path, index=False)
transition_counts


,State,Next_State,scope_label,actual_count
0,0,0.0,0.0,88
1,0,0.0,1.0,5
2,0,0.0,2.0,5
3,0,0.0,3.0,8
4,0,0.0,4.0,15
...,...,...,...,...
16415,9,9.0,25.0,8
16416,9,9.0,26.0,10
16417,9,9.0,27.0,2
16418,9,9.0,28.0,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from re import search
import yfinance as yf
import pandas as pd
import numpy as np

# 나스닥 상위 40개 종목 리스트
nasdaq_tickers = [
    "AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "ADBE", "AVGO",
    "AMD", "INTC", "CSCO", "CMCSA", "PEP", "QCOM", "TXN", "AMAT", "COST", "MRVL",
    "ADP", "VRTX", "ISRG", "INTU", "PYPL", "LRCX", "SBUX", "GILD", "ATVI", "REGN",
    "MDLZ", "BKNG", "SNPS", "ADSK", "ORLY", "KLAC", "FTNT", "PANW", "TEAM", "ZS"
]

# CCI 계산 함수 정의
def calculate_cci(data, n=20):
    """
    특정 종목의 데이터를 받아서 CCI 계산 수행.
    """
    data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3  # Typical Price
    data['SMA_TP'] = data['TP'].rolling(window=n).mean()
    data['MAD'] = data['TP'].rolling(window=n).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    data['CCI'] = (data['TP'] - data['SMA_TP']) / (0.015 * data['MAD'])
    return data

# CCI 상태 라벨링 함수 정의
cci_bins = np.arange(-200, 220, 10)  # 상태 범위
def label_state(value, bins):
    return np.digitize(value, bins, right=True)

# Scope 범위와 라벨 정의
cci_scope_bins = np.arange(-20,20)  # 상태 범위
def label_state_scope(value, bins):
    return np.digitize(value, bins, right=True)

# 모든 종목 데이터를 다운로드하여 결합
data_frames = []
for ticker in nasdaq_tickers:  # 테스트를 위해 5개 종목만 처리
    datay = yf.download(ticker, start="2017-01-01", end="2024-11-24")
    if datay.empty:
        continue
    if isinstance(datay.columns, pd.MultiIndex):
        datay.columns = [col[0] for col in datay.columns.values]
    data_frames.append(datay)
combined_data = pd.concat(data_frames)
combined_data.reset_index(inplace=True)
combined_data.set_index('Date', inplace=True)

# CCI 계산 및 상태 라벨링
data = calculate_cci(combined_data)
data['CCI_State'] = data['CCI'].apply(lambda x: label_state(x, cci_bins))
data['State'] = data['CCI_State'].astype(str)
data['Next_State'] = data['CCI_State'].shift(-1).astype(str)

# Scope 및 Scope Label 계산
data['scope'] = data['CCI_State'] - data['CCI_State'].shift(1)
data['scope_label'] = data['scope'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

# 상태 전이 및 기울기(Scope_Label) 쌍별 count 계산
transition_counts = data.groupby(['State', 'Next_State', 'scope_label']).size().reset_index(name='actual_count')

matrix = pd.DataFrame(transition_counts)
print(matrix)
search_State = "0"
search_inc = 0.0
filter_matrix = matrix.loc[(matrix["State"] == search_State) & (matrix["scope_label"] == search_inc)]



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

      State Next_State  scope_label  actual_count
0         0        0.0          0.0            88
1         0        0.0          1.0             5
2         0        0.0          2.0             5
3         0        0.0          3.0             8
4         0        0.0          4.0            15
...     ...        ...          ...           ...
16415     9        9.0         25.0             8
16416     9        9.0         26.0            10
16417     9        9.0         27.0             2
16418     9        9.0         28.0             1
16419     9        9.0         29.0             3

[16420 rows x 4 columns]


In [ ]:
label_state(-81, cci_bins)

12

In [ ]:
label_state(-103, cci_bins)

10

In [ ]:
label_state_scope(-2, cci_scope_bins)

18

In [ ]:

search_State= "10"
search_inc = 18.0
filter_matrix = matrix.loc[(matrix["State"] == search_State) & (matrix["scope_label"] == search_inc)]
filter_matrix

,State,Next_State,scope_label,actual_count
352,10,0.0,18.0,4
364,10,1.0,18.0,1
378,10,10.0,18.0,11
399,10,11.0,18.0,20
422,10,12.0,18.0,16
443,10,13.0,18.0,9
466,10,14.0,18.0,4
486,10,15.0,18.0,3
505,10,16.0,18.0,5
524,10,17.0,18.0,4


In [ ]:
num_states = len(cci_bins)
start_state = 10
steps= 5
"""
특정 시작 상태에서 종료 상태까지 전이 확률 계산.
"""
P_t = np.zeros((num_states, num_states))
P_t1 = np.zeros((num_states, num_states))

P_t[10] = 1
for _ in range(steps):
  P_t1 = np.zeros(num_states)
  for i in range(num_states):
    for j in range(num_states):
      input = np.zeros(num_states)
      for s in range(num_states):
        scope_sj = j - s
        label_scope_sj = label_state_scope(scope_sj, cci_scope_bins)
        sum_m = filter_matrix.loc[(filter_matrix["State"] == str("여기")) & (filter_matrix["scope_label"] == float(label_scope_sj) )]["actual_count"].sum()
        p_sj = filter_matrix.loc[(filter_matrix["State"] == str(j)) & (filter_matrix["Next_State"] == str(s)) & (filter_matrix["scope_label"] == float(label_scope_sj))]
        input += (p_sj["actual_count"] / sum_m) * P_t[s]
      P_t1[i][j] += input
      pass
  P_t = P_t1


# 전이 확률 계산 함수
#def calculate_transition_probability(start_state, end_cci_state, steps):


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


NameError: name 'f_p_sum' is not defined

In [ ]:
total_sum = filter_matrix['actual_count'].sum()
total_sum

103

In [ ]:
# 상태 전이 입력 확인
print("Transition Counts Sample:")
print(transition_counts.head())  # transition_counts 데이터 확인

# 전이 확률 행렬 입력 확인 (total 방식)
print("\nSample entries from transition_matrix_total:")
non_zero_indices_total = np.argwhere(transition_matrix_total > 0)
for idx in non_zero_indices_total[:10]:  # 최대 10개만 출력
    start_state, next_state, scope_label = idx
    value = transition_matrix_total[start_state, next_state, scope_label]
    print(f"From State {start_state} to {next_state} with scope {scope_label}: {value}")

# 전이 확률 행렬 입력 확인 (scope 방식)
print("\nSample entries from transition_matrix_scope:")
non_zero_indices_scope = np.argwhere(transition_matrix_scope > 0)
for idx in non_zero_indices_scope[:10]:  # 최대 10개만 출력
    start_state, next_state, scope_label = idx
    value = transition_matrix_scope[start_state, next_state, scope_label]
    print(f"From State {start_state} to {next_state} with scope {scope_label}: {value}")

# 전이 행렬 전체 요약
print("\nSummary of transition_matrix_total:")
print(f"Non-zero entries: {len(non_zero_indices_total)}")
print(f"Total sum of transition_matrix_total: {np.sum(transition_matrix_total)}")

print("\nSummary of transition_matrix_scope:")
print(f"Non-zero entries: {len(non_zero_indices_scope)}")
print(f"Total sum of transition_matrix_scope: {np.sum(transition_matrix_scope)}")


Transition Counts Sample:
  State Next_State  scope_label  actual_count
0     0        0.0         30.0             2
1     0        0.0         33.0             2
2     0        0.0         34.0             5
3     0        0.0         35.0             1
4     0        0.0         36.0             5

Sample entries from transition_matrix_total:
From State 0 to 0 with scope 30: 0.0015661707126076742
From State 0 to 0 with scope 33: 0.0015661707126076742
From State 0 to 0 with scope 34: 0.003915426781519186
From State 0 to 0 with scope 35: 0.0007830853563038371
From State 0 to 0 with scope 36: 0.003915426781519186
From State 0 to 0 with scope 37: 0.003915426781519186
From State 0 to 0 with scope 38: 0.007047768206734534
From State 0 to 0 with scope 39: 0.009397024275646046
From State 0 to 0 with scope 40: 0.01096319498825372
From State 0 to 0 with scope 41: 0.007830853563038372

Sample entries from transition_matrix_scope:
From State 0 to 0 with scope 30: 1.0
From State 0 to 0 with scop

In [ ]:
cci_scope_bins

array([-20. , -19.5, -19. , -18.5, -18. , -17.5, -17. , -16.5, -16. ,
       -15.5, -15. , -14.5, -14. , -13.5, -13. , -12.5, -12. , -11.5,
       -11. , -10.5, -10. ,  -9.5,  -9. ,  -8.5,  -8. ,  -7.5,  -7. ,
        -6.5,  -6. ,  -5.5,  -5. ,  -4.5,  -4. ,  -3.5,  -3. ,  -2.5,
        -2. ,  -1.5,  -1. ,  -0.5,   0. ,   0.5,   1. ,   1.5,   2. ,
         2.5,   3. ,   3.5,   4. ,   4.5,   5. ,   5.5,   6. ,   6.5,
         7. ,   7.5,   8. ,   8.5,   9. ,   9.5,  10. ,  10.5,  11. ,
        11.5,  12. ,  12.5,  13. ,  13.5,  14. ,  14.5,  15. ,  15.5,
        16. ,  16.5,  17. ,  17.5,  18. ,  18.5,  19. ,  19.5])

In [ ]:
cci_bins

array([-200, -190, -180, -170, -160, -150, -140, -130, -120, -110, -100,
        -90,  -80,  -70,  -60,  -50,  -40,  -30,  -20,  -10,    0,   10,
         20,   30,   40,   50,   60,   70,   80,   90,  100,  110,  120,
        130,  140,  150,  160,  170,  180,  190,  200,  210])

In [ ]:
label_state(-6, cci_bins)

20

In [ ]:
label_state(-30, cci_bins)

17

In [ ]:
label_state_scope(-3, cci_scope_bins)

17

In [ ]:
datay = yf.download("TSLA", start="2024-05-01", end="2024-11-14")

[*********************100%***********************]  1 of 1 completed


In [ ]:
if isinstance(datay.columns, pd.MultiIndex):
      datay.columns = [col[0] for col in datay.columns.values]

In [ ]:
datay['TP'] = datay["Close"] + datay["High"] + datay["Low"] / 3

In [ ]:
datay

,Adj Close,Close,High,Low,Open,Volume,TP,SMA_TP,MAD,CCI,CCI_State,State,Next_State,scope,scope_label
Date,,,,,,,,,,,,,,,
2018-03-16 00:00:00+00:00,33.000000,33.000000,33.470001,26.059999,27.500000,16931900,30.843334,NaN,NaN,NaN,42,42,42.0,NaN,NaN
2018-03-19 00:00:00+00:00,27.900000,27.900000,31.500000,26.750000,31.450001,5892100,28.716667,NaN,NaN,NaN,42,42,42.0,0.0,40.0
2018-03-20 00:00:00+00:00,30.379999,30.379999,30.903999,28.070000,28.410000,3262000,29.784666,NaN,NaN,NaN,42,42,42.0,0.0,40.0
2018-03-21 00:00:00+00:00,31.080000,31.080000,34.830002,29.290001,30.790001,4460900,31.733334,NaN,NaN,NaN,42,42,42.0,0.0,40.0
2018-03-22 00:00:00+00:00,29.830000,29.830000,31.879999,28.559999,30.000000,3239500,30.090000,NaN,NaN,NaN,42,42,42.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-18 00:00:00+00:00,202.850006,202.850006,204.630005,200.669998,203.270004,1131300,202.716670,192.555267,8.031527,84.345964,29,29,28.0,0.0,40.0
2024-11-19 00:00:00+00:00,204.360001,204.360001,204.970001,198.679993,199.630005,1096200,202.669998,193.357100,8.242443,75.324739,28,28,27.0,-1.0,38.0
2024-11-20 00:00:00+00:00,200.940002,200.940002,205.990005,198.839996,204.000000,1212300,201.923335,194.313100,8.143066,62.304413,27,27,30.0,-1.0,38.0


In [ ]:
datay = calculate_cci(datay)
datay['CCI_State'] = datay['CCI'].apply(lambda x: label_state(x, cci_bins))
datay['State'] = datay['CCI_State'].astype(str)
datay['Next_State'] = datay['CCI_State'].shift(-1).astype(str)

In [ ]:
# Scope 및 Scope Label 계산
datay['scope'] = datay['CCI_State'] - datay['CCI_State'].shift(1)
datay['scope_label'] = datay['scope'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

In [ ]:
import pandas as pd

# transition_counts 데이터를 엑셀로 저장
output_file_path = "/content/datay.xlsx"
datay.to_excel(output_file_path, index=False)
datay

,Adj Close,Close,High,Low,Open,Volume,TP,SMA_TP,MAD,CCI,CCI_State,State,Next_State
Date,,,,,,,,,,,,,
2024-11-01 00:00:00+00:00,248.979996,248.979996,254.000000,246.630005,252.039993,57544800,249.870000,NaN,NaN,NaN,42,42,42.0
2024-11-04 00:00:00+00:00,242.839996,242.839996,248.899994,238.880005,244.559998,68802400,243.539998,NaN,NaN,NaN,42,42,42.0
2024-11-05 00:00:00+00:00,251.440002,251.440002,255.279999,246.210007,247.339996,69282500,250.976669,NaN,NaN,NaN,42,42,42.0
2024-11-06 00:00:00+00:00,288.529999,288.529999,289.589996,275.619995,284.670013,165228700,284.579997,NaN,NaN,NaN,42,42,42.0
2024-11-07 00:00:00+00:00,296.910004,296.910004,299.750000,285.519989,288.890015,117309200,294.059998,NaN,NaN,NaN,42,42,42.0
2024-11-08 00:00:00+00:00,321.220001,321.220001,328.709991,297.660004,299.140015,204782800,315.863332,NaN,NaN,NaN,42,42,42.0
2024-11-11 00:00:00+00:00,350.000000,350.000000,358.640015,336.000000,346.299988,210521600,348.213338,NaN,NaN,NaN,42,42,42.0
2024-11-12 00:00:00+00:00,328.489990,328.489990,345.839996,323.309998,342.739990,155726000,332.546661,NaN,NaN,NaN,42,42,42.0
2024-11-13 00:00:00+00:00,330.239990,330.239990,344.600006,322.500000,335.850006,125405600,332.446665,NaN,NaN,NaN,42,42,nan
